In [ ]:
import numpy as np
import pandas as pd

# import os

# import xgboost as xgb
# from sklearn.linear_model import LogisticRegression

# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss

# from scipy.interpolate import UnivariateSpline
# import statsmodels.api as sm

# import matplotlib.pyplot as plt
# import collections
# import seaborn as sns

# import itertools
# from tqdm import tqdm

# import feature_engine


SEED = 42
CURRENT_YEAR = 2025

In [18]:
DATA_PATH = "/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/DataPreparation/CompData/march-machine-learning-mania-2025/"

In [19]:
Train_Path = "/Users/jimmyhe/Desktop/ML/KaggleCompetitions/NCAA/Train_Set/"
MTeams = pd.read_csv(DATA_PATH + 'MTeams.csv')
WTeams = pd.read_csv(DATA_PATH + "WTeams.csv")
M_flaty = pd.read_csv(Train_Path + "MEN/flaty_elo_M.csv")
M_Kenpom = pd.read_csv(Train_Path + "MEN/Kenpom_final.csv")
M_Massey = pd.read_csv(Train_Path + "MEN/Massey_final.csv")
temp = MTeams[['TeamID', 'LastD1Season']]
M_Kenpom = M_Kenpom.merge(temp, on = "TeamID", how="right")

M_flaty = M_flaty[(M_flaty["LastD1Season"] == 2025) & (M_flaty['Season'] >= 2018)]
M_Kenpom = M_Kenpom[(M_Kenpom["LastD1Season"] == 2025) & (M_Kenpom['Season'] >= 2018)]
M_Massey = M_Massey[(M_Massey["LastD1Season"] == 2025) & (M_Massey['Season'] >= 2018)]
M_Kenpom = M_Kenpom.astype({'Season':'int64','Rk':'int64'})

W_flaty = pd.read_csv(Train_Path + "WOMEN/flaty_elo_W.csv")

M_flaty.shape, M_Kenpom.shape,M_Massey.shape,W_flaty.shape


((2837, 12), (2496, 10), (2486, 28), (9488, 9))

In [20]:
M_Kenpom.head()

,Unnamed: 0,Rk,TeamName,NetRtg,ORtg,DRtg,AdjT,Season,TeamID,LastD1Season
0,218.0,219,abilene chr,-4.46,97.7,102.1,69.4,2025,1101,2025
1,580.0,217,abilene chr,-4.16,101.5,105.6,68.5,2024,1101,2025
2,916.0,191,abilene chr,-1.27,106.0,107.3,69.2,2023,1101,2025
3,1215.0,127,abilene chr,3.59,102.9,99.3,69.9,2022,1101,2025
4,1531.0,85,abilene chr,10.17,103.2,93.0,69.9,2021,1101,2025


In [21]:

M_flaty = M_flaty[['Season', 'TeamID', 'TeamName', 'count', 'win_count', 'gap_avg', 'win_rate', 'Seed', 'Rating']]
M_Kenpom = M_Kenpom[['Season', 'TeamID', 'Rk', 'NetRtg', 'ORtg', 'DRtg', 'AdjT']]
M_Massey = M_Massey[['Season', 'TeamID', 'WIL', 'JNG', '7OT', 'MAS', 'JJK', 'EBP', 'FAS',
                     'HAS', 'DOK', 'LMC', 'PGH', 'BBT', 'BIH', 'TRP', 'DII', 'MOR', 'TRK', 'KPK', 'RT', 'WLK']]  # Removed POM and USA

W_flaty = W_flaty[['Season', 'TeamID', 'TeamName', 'count', 'win_count', 'win_rate', 'gap_avg', 'Seed', 'Rating']]
M_flaty.shape, M_Kenpom.shape,M_Massey.shape,W_flaty.shape



((2837, 9), (2496, 7), (2486, 22), (9488, 9))

In [22]:
merged_df = pd.merge(M_flaty, M_Kenpom, on=['TeamID', 'Season'], how='inner')
final_merged_df = pd.merge(merged_df, M_Massey, on=['TeamID', 'Season'], how='inner')
final_merged_df = final_merged_df[final_merged_df["Season"] <= CURRENT_YEAR]
final_merged_df = final_merged_df.fillna(0) #Fill nan seeds with "0"
final_merged_df.shape

(2486, 34)

In [23]:
final_merged_df.head()

,Season,TeamID,TeamName,count,win_count,gap_avg,win_rate,Seed,Rating,Rk,...,PGH,BBT,BIH,TRP,DII,MOR,TRK,KPK,RT,WLK
0,2025,1120,Auburn,33,28,14.242424,0.848485,1.0,65.690297,4,...,1,2,1,2,1,4,3,1,2,2
1,2025,1181,Duke,34,31,20.794118,0.911765,1.0,66.375404,1,...,4,1,3,1,2,1,2,2,1,1
2,2025,1196,Florida,34,30,16.176471,0.882353,1.0,66.166661,2,...,2,3,2,4,3,2,4,4,5,3
3,2025,1222,Houston,34,30,15.735294,0.882353,1.0,68.295069,3,...,3,4,4,3,4,3,1,3,3,4
4,2025,1104,Alabama,33,25,9.696970,0.757576,2.0,64.908072,6,...,5,6,6,5,5,5,5,5,6,5


In [24]:
final_merged_df.to_csv('DataSource.csv')

In [25]:
submission_file = pd.read_csv(DATA_PATH + 'SampleSubmissionStage2.csv')
MTourney = pd.read_csv(DATA_PATH + 'MNCAATourneyCompactResults.csv')

In [26]:

tmp = MTourney.copy()
def generate_team_concat_and_target(row):
    lower_id = min(row['WTeamID'], row['LTeamID'])
    higher_id = max(row['WTeamID'], row['LTeamID'])

    team_concat = f"{row['Season']}_{lower_id}_{higher_id}"
    
    target = int(row['WTeamID'] == lower_id)

    
    return pd.Series([team_concat, target, lower_id, higher_id, row['Season']])

# Apply function to each row
tmp[['ID', 'Target', 'FirstID', 'SecondID', 'Season']] = MTourney.apply(generate_team_concat_and_target, axis=1)
tmp = tmp[tmp["Season"] >= 2018]
target_df = tmp[['ID','Target','FirstID', 'SecondID', 'Season']]
target_df.tail()

,ID,Target,FirstID,SecondID,Season
2513,2024_1181_1301,0,1181,1301,2024
2514,2024_1345_1397,1,1345,1397,2024
2515,2024_1104_1163,0,1104,1163,2024
2516,2024_1301_1345,0,1301,1345,2024
2517,2024_1163_1345,1,1163,1345,2024


In [27]:
#This is for training

def create_merged_features(target_df, features_df):
    # Merge Team1 features
    merged = pd.merge(
        target_df,
        features_df.add_prefix('Team1_'),
        left_on=['Season', 'FirstID'],
        right_on=['Team1_Season', 'Team1_TeamID'],
        how='left'
    )
    
    # Merge Team2 features
    merged = pd.merge(
        merged,
        features_df.add_prefix('Team2_'),
        left_on=['Season', 'SecondID'],
        right_on=['Team2_Season', 'Team2_TeamID'],
        how='left'
    )
    
    # Clean up merge columns
    merged = merged.drop(columns=[
        'Team1_Season', 'Team1_TeamID', 
        'Team2_Season', 'Team2_TeamID'
    ])
    
    return merged

# Usage
concatenated_features_df = create_merged_features(target_df, final_merged_df)
concatenated_features_df = concatenated_features_df[concatenated_features_df['ID'].astype(str) != '2021_1124_1216']
concatenated_features_df.to_csv('concatenated_features_df.csv')
concatenated_features_df

,ID,Target,FirstID,SecondID,Season,Team1_TeamName,Team1_count,Team1_win_count,Team1_gap_avg,Team1_win_rate,...,Team2_PGH,Team2_BBT,Team2_BIH,Team2_TRP,Team2_DII,Team2_MOR,Team2_TRK,Team2_KPK,Team2_RT,Team2_WLK
0,2018_1254_1347,0,1254,1347,2018,LIU Brooklyn,33,17,-0.121212,0.515152,...,165.0,160.0,163.0,187.0,173.0,165.0,178.0,154.0,182,170.0
1,2018_1382_1417,1,1382,1417,2018,St Bonaventure,32,25,6.906250,0.781250,...,34.0,58.0,46.0,46.0,42.0,54.0,46.0,43.0,52,44.0
2,2018_1113_1393,0,1113,1393,2018,Arizona St,31,20,8.193548,0.645161,...,52.0,60.0,55.0,55.0,54.0,44.0,52.0,54.0,64,50.0
3,2018_1300_1411,0,1300,1411,2018,NC Central,31,16,-0.322581,0.516129,...,255.0,228.0,277.0,238.0,264.0,256.0,264.0,219.0,274,240.0
4,2018_1104_1439,1,1104,1439,2018,Alabama,34,19,2.382353,0.558824,...,42.0,36.0,34.0,36.0,27.0,32.0,30.0,33.0,28,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,2024_1181_1301,0,1181,1301,2024,Duke,32,24,12.406250,0.750000,...,46.0,96.0,71.0,60.0,54.0,43.0,63.0,59.0,67,59.0
397,2024_1345_1397,1,1345,1397,2024,Purdue,33,29,13.242424,0.878788,...,6.0,4.0,4.0,6.0,5.0,7.0,7.0,6.0,8,6.0
398,2024_1104_1163,0,1104,1163,2024,Alabama,32,21,9.687500,0.656250,...,2.0,3.0,2.0,2.0,1.0,2.0,2.0,3.0,4,1.0
399,2024_1301_1345,0,1301,1345,2024,NC State,36,22,3.666667,0.611111,...,1.0,1.0,1.0,3.0,2.0,6.0,3.0,1.0,3,3.0


In [28]:
concatenated_features_df.columns

Index(['ID', 'Target', 'FirstID', 'SecondID', 'Season', 'Team1_TeamName',
       'Team1_count', 'Team1_win_count', 'Team1_gap_avg', 'Team1_win_rate',
       'Team1_Seed', 'Team1_Rating', 'Team1_Rk', 'Team1_NetRtg', 'Team1_ORtg',
       'Team1_DRtg', 'Team1_AdjT', 'Team1_WIL', 'Team1_JNG', 'Team1_7OT',
       'Team1_MAS', 'Team1_JJK', 'Team1_EBP', 'Team1_FAS', 'Team1_HAS',
       'Team1_DOK', 'Team1_LMC', 'Team1_PGH', 'Team1_BBT', 'Team1_BIH',
       'Team1_TRP', 'Team1_DII', 'Team1_MOR', 'Team1_TRK', 'Team1_KPK',
       'Team1_RT', 'Team1_WLK', 'Team2_TeamName', 'Team2_count',
       'Team2_win_count', 'Team2_gap_avg', 'Team2_win_rate', 'Team2_Seed',
       'Team2_Rating', 'Team2_Rk', 'Team2_NetRtg', 'Team2_ORtg', 'Team2_DRtg',
       'Team2_AdjT', 'Team2_WIL', 'Team2_JNG', 'Team2_7OT', 'Team2_MAS',
       'Team2_JJK', 'Team2_EBP', 'Team2_FAS', 'Team2_HAS', 'Team2_DOK',
       'Team2_LMC', 'Team2_PGH', 'Team2_BBT', 'Team2_BIH', 'Team2_TRP',
       'Team2_DII', 'Team2_MOR', 'Team2_TRK',

In [29]:
non_feature_cols = ['ID', 'Target', 'FirstID', 'SecondID', 'Season', 
                   'Team1_TeamName', 'Team2_TeamName']

features = [col for col in concatenated_features_df.columns 
           if col not in non_feature_cols]

concatenated_features_df = concatenated_features_df[concatenated_features_df['ID'].astype(str) != '2021_1124_1216']
##So we are met with an issue that Hartford (ID: 1216) stopped participating since 2023, so no data is contained for this one
test_set  = concatenated_features_df[concatenated_features_df['Season']==2024] 
train_set = concatenated_features_df[concatenated_features_df['Season'] != 2024]
X = train_set[features]
y = train_set['Target']
X_test = test_set[features]
y_test = test_set['Target']
X.shape, y.shape, X_test.shape, y_test.shape

((333, 62), (333,), (67, 62), (67,))

In [30]:
rows_with_missing = concatenated_features_df[concatenated_features_df.isna().any(axis=1)]
len(rows_with_missing)


0

In [31]:
from sklearn.linear_model import LogisticRegression

lr_mens = LogisticRegression()
lr_mens.fit(X, y)

#?????


/opt/miniconda3/envs/NCAA/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [32]:
from sklearn.preprocessing import StandardScaler  # Resistant to outliers
scaler = StandardScaler()
X_final_scaled = scaler.fit_transform(X_final)


from sklearn.linear_model import LogisticRegressionCV  # Built-in cross-validation

model = LogisticRegressionCV(
    Cs=10,  # Test 10 regularization strengths
    penalty='l1',  # LASSO for feature selection
    solver='liblinear',
    scoring='neg_log_loss',
    class_weight='balanced',
    max_iter=1000,
    cv=5
)
model.fit(X_final_scaled, y)

# Get the mean cross-validated score (negative log loss)
mean_score = model.scores_[1].mean(axis=0)

# The scores are negative, so we negate them to get the actual log loss
log_loss = -mean_score

print(f"Mean Log Loss: {log_loss}")

# If you want to see scores for different regularization strengths:
for i, C in enumerate(model.Cs_):
    print(f"C={C:.4f}, Log Loss: {-mean_score[i]:.4f}")


NameError: name 'X_final' is not defined

In [ ]:
import pandas as pd
import numpy as np

def create_differential_features(features_df):
    # Get columns to subtract (excluding TeamName)
    cols_to_subtract = [col for col in features_df.columns if col.startswith('Team1_') and col != 'Team1_TeamName']
    
    for col in cols_to_subtract:
        team2_col = col.replace('Team1_', 'Team2_')
        diff_col = col.replace('Team1_', 'Diff_')
        
        # Convert to numeric, coerce errors to NaN
        features_df[col] = pd.to_numeric(features_df[col], errors='coerce')
        features_df[team2_col] = pd.to_numeric(features_df[team2_col], errors='coerce')
        
        # Perform subtraction, handling NaN values
        features_df[diff_col] = features_df[col].subtract(features_df[team2_col], fill_value=np.nan)
    
    cols_to_keep = ['ID', 'Target', 'FirstID', 'SecondID', 'Season']
    # Select columns to keep
    cols_to_keep = (cols_to_keep + 
                    ['Team1_TeamName', 'Team2_TeamName'] + 
                    [col for col in features_df.columns if col.startswith('Diff_')])
    
    # Create final dataframe
    result = features_df[cols_to_keep]
    
    return result

# Usage
dif_features_df = create_differential_features(concatenated_features_df)
dif_features_df.to_csv('dif_features_df.csv')
dif_features_df

,ID,Target,FirstID,SecondID,Season,Team1_TeamName,Team2_TeamName,Diff_count,Diff_win_count,Diff_gap_avg,...,Diff_PGH,Diff_BBT,Diff_BIH,Diff_TRP,Diff_DII,Diff_MOR,Diff_TRK,Diff_KPK,Diff_RT,Diff_WLK
0,2018_1254_1347,0,1254,1347,2018,LIU Brooklyn,Radford,1.0,-3.0,-2.183712,...,84.0,83.0,78.0,72.0,70.0,40.0,77.0,79.0,54,72.0
1,2018_1382_1417,1,1382,1417,2018,St Bonaventure,UCLA,0.0,4.0,1.312500,...,10.0,-7.0,-10.0,21.0,19.0,6.0,24.0,-6.0,1,15.0
2,2018_1113_1393,0,1113,1393,2018,Arizona St,Syracuse,-2.0,0.0,5.193548,...,-4.0,-6.0,2.0,-7.0,-1.0,8.0,-2.0,-9.0,-19,-7.0
3,2018_1300_1411,0,1300,1411,2018,NC Central,TX Southern,-3.0,1.0,1.765655,...,31.0,61.0,15.0,58.0,39.0,54.0,51.0,48.0,22,56.0
4,2018_1104_1439,1,1104,1439,2018,Alabama,Virginia Tech,2.0,-2.0,-5.586397,...,-1.0,29.0,19.0,9.0,22.0,8.0,18.0,16.0,33,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,2024_1181_1301,0,1181,1301,2024,Duke,NC State,-4.0,2.0,8.739583,...,-31.0,-88.0,-56.0,-50.0,-44.0,-32.0,-52.0,-47.0,-52,-49.0
397,2024_1345_1397,1,1345,1397,2024,Purdue,Tennessee,1.0,5.0,1.648674,...,-5.0,-3.0,-3.0,-3.0,-3.0,-1.0,-4.0,-5.0,-5,-3.0
398,2024_1104_1163,0,1104,1163,2024,Alabama,Connecticut,-2.0,-10.0,-7.371324,...,14.0,12.0,16.0,6.0,11.0,12.0,13.0,7.0,3,12.0
399,2024_1301_1345,0,1301,1345,2024,NC State,Purdue,3.0,-7.0,-9.575758,...,45.0,95.0,70.0,57.0,52.0,37.0,60.0,58.0,64,56.0


In [ ]:
dif_features_df.isna().sum().sort_values(ascending=False)


ID                0
Diff_PGH          0
Diff_MAS          0
Diff_JJK          0
Diff_EBP          0
Diff_FAS          0
Diff_HAS          0
Diff_DOK          0
Diff_LMC          0
Diff_BBT          0
Target            0
Diff_BIH          0
Diff_TRP          0
Diff_DII          0
Diff_MOR          0
Diff_TRK          0
Diff_KPK          0
Diff_RT           0
Diff_7OT          0
Diff_JNG          0
Diff_WIL          0
Diff_AdjT         0
FirstID           0
SecondID          0
Season            0
Team1_TeamName    0
Team2_TeamName    0
Diff_count        0
Diff_win_count    0
Diff_gap_avg      0
Diff_win_rate     0
Diff_Seed         0
Diff_Rating       0
Diff_Rk           0
Diff_NetRtg       0
Diff_ORtg         0
Diff_DRtg         0
Diff_WLK          0
dtype: int64

In [ ]:
dif_features_df.shape

(400, 38)

In [ ]:
dif_features_df.columns

Index(['ID', 'Target', 'FirstID', 'SecondID', 'Season', 'Team1_TeamName',
       'Team2_TeamName', 'Diff_count', 'Diff_win_count', 'Diff_gap_avg',
       'Diff_win_rate', 'Diff_Seed', 'Diff_Rating', 'Diff_Rk', 'Diff_NetRtg',
       'Diff_ORtg', 'Diff_DRtg', 'Diff_AdjT', 'Diff_WIL', 'Diff_JNG',
       'Diff_7OT', 'Diff_MAS', 'Diff_JJK', 'Diff_EBP', 'Diff_FAS', 'Diff_HAS',
       'Diff_DOK', 'Diff_LMC', 'Diff_PGH', 'Diff_BBT', 'Diff_BIH', 'Diff_TRP',
       'Diff_DII', 'Diff_MOR', 'Diff_TRK', 'Diff_KPK', 'Diff_RT', 'Diff_WLK'],
      dtype='object')

In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, mean_absolute_error

# Split data
cols_to_drop = ['ID', 'Target', 'FirstID', 'SecondID', 'Season', 'Team1_TeamName','Team2_TeamName']
X_diff = dif_features_df.drop(columns=cols_to_drop, axis=1)
y_diff = dif_features_df['Target']
X_train_diff, X_test_diff, y_train_diff, y_test_diff = train_test_split(X, y, test_size=0.2, random_state=SEED)


model = RandomForestClassifier(
    n_estimators=100,
    oob_score=True,  # Enable out-of-bag estimates
    random_state=42
)
model.fit(X_train_diff, y_train_diff)
y_train_pred = model.predict(X_train_diff)
train_mae = mean_absolute_error(y_train_diff, y_train_pred)

# Calculate MAE for test set
y_test_pred = model.predict(X_test)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"Training MAE: {train_mae:.4f}")
print(f"Test MAE: {test_mae:.4f}")
important_features = X.columns[model.feature_importances_ > model.feature_importances_.mean()]


Training MAE: 0.0000
Test MAE: 0.2985


In [ ]:
optimized_model = RandomForestClassifier()
optimized_model.fit(X_train_diff[important_features], y_train_diff)
y_train_pred = optimized_model.predict(X_train_diff[important_features])
train_mae = mean_absolute_error(y_train_diff, y_train_pred)
y_test_pred = optimized_model.predict(X_test_diff[important_features])
test_mae = mean_absolute_error(y_test_diff, y_test_pred)

print(f"Training MAE: {train_mae:.4f}")
print(f"Test MAE: {test_mae:.4f}")


Training MAE: 0.0000
Test MAE: 0.4030


In [ ]:
pred = model.predict(X_test_diff)


In [33]:
WTourney = pd.read_csv(DATA_PATH + 'WNCAATourneyCompactResults.csv')

tmp = MTourney.copy()
def generate_team_concat_and_target(row):
    lower_id = min(row['WTeamID'], row['LTeamID'])
    higher_id = max(row['WTeamID'], row['LTeamID'])

    team_concat = f"{row['Season']}_{lower_id}_{higher_id}"
    
    target = int(row['WTeamID'] == lower_id)

    
    return pd.Series([team_concat, target, lower_id, higher_id, row['Season']])

# Apply function to each row
tmp[['ID', 'Target', 'FirstID', 'SecondID', 'Season']] = MTourney.apply(generate_team_concat_and_target, axis=1)
tmp = tmp[tmp["Season"] >= 2018]
W_target_df = tmp[['ID','Target','FirstID', 'SecondID', 'Season']]
W_target_df.tail()
W_target_df.to_csv('WTarget.csv')